In [ ]:
# %pip install python-dotenv
# %pip install geopandas

In [1]:
import os
import dotenv
import geopandas as gpd

dotenv.load_dotenv()

geopackage_name = 'output/wind-turbines.gpkg'

In [2]:
wind_turbines = gpd.read_file(os.path.join(os.environ['DATA_DIR'], 'WindTurbines/uswtdbSHP/uswtdb_v6_1_20231128.shp'))
wind_turbines = wind_turbines.to_crs('EPSG:4326')
wind_turbines.to_file(geopackage_name, layer='wind_turbines', driver='GPKG')

In [3]:
countries_50m = gpd.read_file(os.path.join(os.environ['NATURAL_EARTH_DIR'], 'Countries/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp'))
states_provinces_50m = gpd.read_file(os.path.join(os.environ['NATURAL_EARTH_DIR'], 'StatesProvinces/ne_50m_admin_1_states_provinces/ne_50m_admin_1_states_provinces.shp'))
lakes_50m = gpd.read_file(os.path.join(os.environ['NATURAL_EARTH_DIR'], 'Lakes/ne_50m_lakes/ne_50m_lakes.shp'))

In [4]:
united_states_50m = countries_50m[countries_50m['ADMIN'] == 'United States of America']
us_states_50m = states_provinces_50m[states_provinces_50m['iso_a2'] == 'US']
us_states_50m = us_states_50m.join(gpd.sjoin(wind_turbines, us_states_50m).groupby("index_right").size().rename("wind_turbine_join_count"), how="left")
us_states_50m['wind_turbine_join_count'] = us_states_50m['wind_turbine_join_count'].fillna(0).astype(int)
great_lakes_50m = lakes_50m[lakes_50m['name'].isin(['Lake Superior', 'Lake Michigan', 'Lake Huron', 'Lake Erie', 'Lake Ontario'])]
states_with_no_wind_turbines_50m = us_states_50m[us_states_50m['wind_turbine_join_count'] == 0]
states_with_at_least_one_wind_turbine_50m = us_states_50m[us_states_50m['wind_turbine_join_count'] > 0]

In [5]:
united_states_50m.to_file(geopackage_name, layer='united_states_50m', driver='GPKG')
us_states_50m.to_file(geopackage_name, layer='us_states_50m', driver='GPKG')
great_lakes_50m.to_file(geopackage_name, layer='great_lakes_50m', driver='GPKG')
states_with_no_wind_turbines_50m.to_file(geopackage_name, layer='states_with_no_wind_turbines_50m', driver='GPKG')
states_with_at_least_one_wind_turbine_50m.to_file(geopackage_name, layer='states_with_at_least_one_wind_turbine_50m', driver='GPKG')